In [871]:
from importlib import reload

import dnd
reload(dnd)
import pandas as pd
from dnd import DiceRoll as dr

import numpy as np

In [876]:
level = 6
AS = 2
melee_weapon = dr(1,6)
smite = dr(2,8)
to_hit = dnd.proficency_bonus(level)+AS
bonus_damage = AS

pally_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage=bonus_damage, crit_added_damage_dice=smite)
pally_attack2 = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage, crit_added_damage_dice=smite)
pam_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(1,4), bonus_damage, crit_added_damage_dice=smite)

lvl_6_pally = pally_attack + pally_attack2 + pam_attack

wis_saving_throw = dnd.spell_save(-1 + 3 + dnd.proficency_bonus(level), 10+level/2)
dex_saving_throw = dnd.spell_save(-1 + 3, 10+level/2)

chance_to_be_hit = (dnd.roll_damage(21, dnd.attack_bonus(level), dr(1,2)) > 0).mean()

print('Chace to be hit', chance_to_be_hit)
print('Wisdom saving throw:', wis_saving_throw.mean())
print('Dex saving throw:',dex_saving_throw.mean())

print('Damage per round:', lvl_6_pally.mean())

Chace to be hit 0.3523
Wisdom saving throw: 0.6516
Dex saving throw: 0.4975
Damage per round: 10.3398


In [883]:
level = 7
AS = 2
melee_weapon = dr(1,6)
smite = dr(2,8)
to_hit = dnd.proficency_bonus(level)+AS
bonus_damage = AS+3

pally_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage=bonus_damage, crit_added_damage_dice=smite)
pally_attack2 = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage, crit_added_damage_dice=smite)
pam_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(1,4), bonus_damage, crit_added_damage_dice=smite)

wis_saving_throw = dnd.spell_save(-1 + AS + dnd.proficency_bonus(level), 10+level/2)
dex_saving_throw = dnd.spell_save(-1 + AS, 10+level/2)

chance_to_be_hit = (dnd.roll_damage(21, dnd.attack_bonus(level), dr(1,2)) > 0).mean()

print('Chace to be hit', chance_to_be_hit)
print('Wisdom saving throw:', wis_saving_throw.mean())
print('Dex saving throw:',dex_saving_throw.mean())

print('Damage for first round:', (pally_attack + pally_attack2).mean())
print('Damage for second round:', (pally_attack + pally_attack2 + pam_attack).mean())

Chace to be hit 0.3471
Wisdom saving throw: 0.5519
Dex saving throw: 0.3899
Damage for first round: 10.6975
Damage for second round: 15.443


In [884]:
level = 7
AS = 3
melee_weapon = dr(2,6)
smite = dr(2,8)
to_hit = dnd.proficency_bonus(level)+AS
bonus_damage = AS
adv = {}#{'advantage':True, 'crit_on':19}

pally_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage=bonus_damage, crit_added_damage_dice=smite, **adv)
eldrith_blast = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(2,10)+dr(1,6))
pally_attack2 = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage, crit_added_damage_dice=smite, **adv)
pam_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(1,4), bonus_damage, crit_added_damage_dice=smite, **adv)

wis_saving_throw = dnd.spell_save(-1 + AS + dnd.proficency_bonus(level), 10+level/2)
dex_saving_throw = dnd.spell_save(-1 + AS, 10+level/2)

chance_to_be_hit = (dnd.roll_damage(21, dnd.attack_bonus(level), dr(1,2)) > 0).mean()

print('Chace to be hit', chance_to_be_hit)
print('Wisdom saving throw:', wis_saving_throw.mean())
print('Dex saving throw:',dex_saving_throw.mean())

print('Edritch blast round:', eldrith_blast.mean())
print('Damage for first round:', (pally_attack + pally_attack2).mean())
print('Damage for second round:', (pally_attack + pally_attack2 + pam_attack).mean())

Chace to be hit 0.344
Wisdom saving throw: 0.5979
Dex saving throw: 0.45
Edritch blast round: 9.2859
Damage for first round: 13.6039
Damage for second round: 17.4907


In [885]:
level = 8
AS = 3
melee_weapon = dr(2,6)
smite = dr(2,8)
to_hit = dnd.proficency_bonus(level)+AS
bonus_damage = AS*2
adv = {}#{'advantage':True, 'crit_on':19}

pally_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage=bonus_damage, crit_added_damage_dice=smite, **adv)
eldrith_blast = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(2,10)+dr(1,6))
pally_attack2 = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage, crit_added_damage_dice=smite, **adv)
pam_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(1,4), bonus_damage, crit_added_damage_dice=smite, **adv)

wis_saving_throw = dnd.spell_save(-1 + AS + dnd.proficency_bonus(level), 10+level/2)
dex_saving_throw = dnd.spell_save(-1 + AS, 10+level/2)

chance_to_be_hit = (dnd.roll_damage(21, dnd.attack_bonus(level), dr(1,2)) > 0).mean()

print('Chace to be hit', chance_to_be_hit)
print('Wisdom saving throw:', wis_saving_throw.mean())
print('Dex saving throw:',dex_saving_throw.mean())

print('Edritch blast round:', eldrith_blast.mean())
print('Damage for first round:', (pally_attack + pally_attack2).mean())
print('Damage for second round:', (pally_attack + pally_attack2 + pam_attack).mean())

Chace to be hit 0.4059
Wisdom saving throw: 0.6025
Dex saving throw: 0.4451
Edritch blast round: 8.8431
Damage for first round: 15.8056
Damage for second round: 21.1745


In [886]:
level = 9
AS = 4
melee_weapon = dr(2,6)
smite = dr(2,8)
to_hit = dnd.proficency_bonus(level)+AS
bonus_damage = AS*2
adv = {}#{'advantage':True, 'crit_on':19}

pally_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage=bonus_damage, crit_added_damage_dice=smite, **adv)
eldrith_blast = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(2,10)+dr(2,6))
pally_attack2 = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage, crit_added_damage_dice=smite, **adv)
pam_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(1,4), bonus_damage, crit_added_damage_dice=smite, **adv)

wis_saving_throw = dnd.spell_save(-1 + AS + dnd.proficency_bonus(level), 10+level/2)
dex_saving_throw = dnd.spell_save(-1 + AS, 10+level/2)

chance_to_be_hit = (dnd.roll_damage(21, dnd.attack_bonus(level), dr(1,2)) > 0).mean()

print('Chace to be hit', chance_to_be_hit)
print('Wisdom saving throw:', wis_saving_throw.mean())
print('Dex saving throw:',dex_saving_throw.mean())

print('Edritch blast round:', eldrith_blast.mean())
print('Damage for first round:', (pally_attack + pally_attack2).mean())
print('Damage for second round:', (pally_attack + pally_attack2 + pam_attack).mean())

Chace to be hit 0.4611
Wisdom saving throw: 0.6478
Dex saving throw: 0.4489
Edritch blast round: 12.6614
Damage for first round: 21.0933
Damage for second round: 28.5174


In [887]:
level = 10
AS = 4
melee_weapon = dr(2,6)
smite = dr(2,8)
to_hit = dnd.proficency_bonus(level)+AS
bonus_damage = AS*2
adv = {}#{'advantage':True, 'crit_on':19}

pally_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage=bonus_damage, crit_added_damage_dice=smite, **adv)
eldrith_blast = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(2,10)+dr(2,6), bonus_damage=AS*2) 
pally_attack2 = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage, crit_added_damage_dice=smite, **adv)
pam_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(1,4), bonus_damage, crit_added_damage_dice=smite, **adv)

wis_saving_throw = dnd.spell_save(-1 + AS + dnd.proficency_bonus(level), 10+level/2)
dex_saving_throw = dnd.spell_save(-1 + AS, 10+level/2)

chance_to_be_hit = (dnd.roll_damage(21, dnd.attack_bonus(level), dr(1,2)) > 0).mean()

print('Chace to be hit', chance_to_be_hit)
print('Wisdom saving throw:', wis_saving_throw.mean())
print('Dex saving throw:',dex_saving_throw.mean())

print('Edritch blast round:', eldrith_blast.mean())
print('Damage for first round:', (pally_attack + pally_attack2).mean())
print('Damage for second round:', (pally_attack + pally_attack2 + pam_attack).mean())

Chace to be hit 0.4593
Wisdom saving throw: 0.65
Dex saving throw: 0.446
Edritch blast round: 16.5467
Damage for first round: 19.5937
Damage for second round: 26.5229


In [888]:
level = 14
AS = 5
melee_weapon = dr(2,6)+dr(1,8)
smite = dr(2,8)
to_hit = dnd.proficency_bonus(level)+AS
bonus_damage = AS*2
adv = {}#{'advantage':True, 'crit_on':19}

pally_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage=bonus_damage, crit_added_damage_dice=smite, **adv)
eldrith_blast = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(2,10)+dr(2,6), bonus_damage=AS*2) 
pally_attack2 = dnd.roll_damage(dnd.get_AC(level), to_hit, melee_weapon, bonus_damage, crit_added_damage_dice=smite, **adv)
pam_attack = dnd.roll_damage(dnd.get_AC(level), to_hit, dr(1,4), bonus_damage, crit_added_damage_dice=smite, **adv)

wis_saving_throw = dnd.spell_save(-1 + AS + dnd.proficency_bonus(level), 10+level/2)
dex_saving_throw = dnd.spell_save(-1 + AS, 10+level/2)

chance_to_be_hit = (dnd.roll_damage(21, dnd.attack_bonus(level), dr(1,2)) > 0).mean()

print('Chace to be hit', chance_to_be_hit)
print('Wisdom saving throw:', wis_saving_throw.mean())
print('Dex saving throw:',dex_saving_throw.mean())

print('Edritch blast round:', eldrith_blast.mean())
print('Damage for first round:', (pally_attack + pally_attack2).mean())
print('Damage for second round:', (pally_attack + pally_attack2 + pam_attack).mean())

Chace to be hit 0.4993
Wisdom saving throw: 0.6573
Dex saving throw: 0.3915
Edritch blast round: 18.8401
Damage for first round: 29.9548
Damage for second round: 38.6185
